In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Add, MaxPooling2D, Flatten, Dense, Softmax, Activation, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import Adam, Nadam, SGD
import numpy as np
import h5py

In [2]:
tf.__version__
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
def dataGenerator(start_index, stop_index):
    def generator():
        indexes = shuff[start_index : stop_index]
        for i in indexes:
            x = np.array(hdf5['Xs'][i])
            y = np.array(hdf5['Ys'][i])
            yield x, y
    return generator

In [4]:
hdf5 = h5py.File('./hdf5/kyu_dataset_cor.hdf5','r')
batch_size = 64
dataset_size = hdf5['Xs'].shape[0]
val_start = 0.8
val_stop = 1.0

train_steps = int(dataset_size * (val_start)/batch_size)
val_steps = int(dataset_size * (val_stop - val_start)/batch_size)
print(f"train_steps: {train_steps}, val_steps: {val_steps}")
shuff = np.arange(dataset_size)
np.random.shuffle(shuff)

data_gen = dataGenerator(start_index = 0, stop_index = int(dataset_size * val_start)) # 90% of the complete dataset
dataset = tf.data.Dataset.from_generator(data_gen, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,19)),tf.TensorShape((1))))
dataset = dataset.batch(batch_size, drop_remainder=True)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

data_gen_valid = dataGenerator(start_index=int(dataset_size * val_start), # 10% of the complete dataset
                               stop_index=int(dataset_size * val_stop))
dataset_valid = tf.data.Dataset.from_generator(data_gen_valid, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,19)),tf.TensorShape((1))))
dataset_valid = dataset_valid.batch(batch_size, drop_remainder=True)
dataset_valid = dataset_valid.prefetch(tf.data.AUTOTUNE)


train_steps: 169592, val_steps: 42398


In [5]:
# resnet50
def block1(x, filters, kernel_size=3, stride=1, conv_shortcut=False):
    bn_axis=3
    preact = Activation("relu")(x)
    if conv_shortcut:
        shortcut = Conv2D(4 * filters, 1, strides=stride)(preact)
    else:
        shortcut = x
    
    x = Conv2D(filters, 1, strides=1, use_bias=False)(preact)
    x = Activation("relu")(x)
    x = ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = Conv2D(
        filters,
        kernel_size,
        strides=stride,
        use_bias=False
    )(x)
    x = Activation("relu")(x)
    x = Conv2D(4 * filters, 1)(x)
    x = Add()([shortcut, x])
    return x

def ResNet50(include_top=True,
             input_tensor=None,
             input_shape=None,
             pooling=False):
    inputs = Input(shape = input_shape)
    x = block1(inputs, 256, conv_shortcut=True)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(361, activation='softmax', name='fc1000')(x)

    model = Model(inputs, x, name='resnet50')
    return model

model = ResNet50(include_top=True,
                 input_tensor=None,
                 input_shape=(19, 19, 19),
                 pooling=False)
opt = Nadam(learning_rate = 0.0001)
model.compile(optimizer = opt,
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 19, 19, 19)  0           []                               
                                ]                                                                 
                                                                                                  
 activation (Activation)        (None, 19, 19, 19)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 19, 19, 256)  4864        ['activation[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 19, 19, 256)  0           ['conv2d_1[0][0]']        

                                                                                                  
 add_3 (Add)                    (None, 19, 19, 1024  0           ['add_2[0][0]',                  
                                )                                 'conv2d_12[0][0]']              
                                                                                                  
 activation_12 (Activation)     (None, 19, 19, 1024  0           ['add_3[0][0]']                  
                                )                                                                 
                                                                                                  
 conv2d_13 (Conv2D)             (None, 19, 19, 256)  262144      ['activation_12[0][0]']          
                                                                                                  
 activation_13 (Activation)     (None, 19, 19, 256)  0           ['conv2d_13[0][0]']              
          

                                                                                                  
 activation_24 (Activation)     (None, 19, 19, 1024  0           ['add_7[0][0]']                  
                                )                                                                 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 19, 19, 256)  262144      ['activation_24[0][0]']          
                                                                                                  
 activation_25 (Activation)     (None, 19, 19, 256)  0           ['conv2d_25[0][0]']              
                                                                                                  
 zero_padding2d_8 (ZeroPadding2  (None, 21, 21, 256)  0          ['activation_25[0][0]']          
 D)                                                                                               
          

                                                                                                  
 conv2d_37 (Conv2D)             (None, 19, 19, 256)  262144      ['activation_36[0][0]']          
                                                                                                  
 activation_37 (Activation)     (None, 19, 19, 256)  0           ['conv2d_37[0][0]']              
                                                                                                  
 zero_padding2d_12 (ZeroPadding  (None, 21, 21, 256)  0          ['activation_37[0][0]']          
 2D)                                                                                              
                                                                                                  
 conv2d_38 (Conv2D)             (None, 19, 19, 256)  589824      ['zero_padding2d_12[0][0]']      
                                                                                                  
 activatio

 activation_49 (Activation)     (None, 19, 19, 256)  0           ['conv2d_49[0][0]']              
                                                                                                  
 zero_padding2d_16 (ZeroPadding  (None, 21, 21, 256)  0          ['activation_49[0][0]']          
 2D)                                                                                              
                                                                                                  
 conv2d_50 (Conv2D)             (None, 19, 19, 256)  589824      ['zero_padding2d_16[0][0]']      
                                                                                                  
 activation_50 (Activation)     (None, 19, 19, 256)  0           ['conv2d_50[0][0]']              
                                                                                                  
 conv2d_51 (Conv2D)             (None, 19, 19, 1024  263168      ['activation_50[0][0]']          
          

 2D)                                                                                              
                                                                                                  
 conv2d_62 (Conv2D)             (None, 19, 19, 256)  589824      ['zero_padding2d_20[0][0]']      
                                                                                                  
 activation_62 (Activation)     (None, 19, 19, 256)  0           ['conv2d_62[0][0]']              
                                                                                                  
 conv2d_63 (Conv2D)             (None, 19, 19, 1024  263168      ['activation_62[0][0]']          
                                )                                                                 
                                                                                                  
 add_20 (Add)                   (None, 19, 19, 1024  0           ['add_19[0][0]',                 
          

                                                                                                  
 activation_74 (Activation)     (None, 19, 19, 256)  0           ['conv2d_74[0][0]']              
                                                                                                  
 conv2d_75 (Conv2D)             (None, 19, 19, 1024  263168      ['activation_74[0][0]']          
                                )                                                                 
                                                                                                  
 add_24 (Add)                   (None, 19, 19, 1024  0           ['add_23[0][0]',                 
                                )                                 'conv2d_75[0][0]']              
                                                                                                  
 activation_75 (Activation)     (None, 19, 19, 1024  0           ['add_24[0][0]']                 
          

 conv2d_87 (Conv2D)             (None, 19, 19, 1024  263168      ['activation_86[0][0]']          
                                )                                                                 
                                                                                                  
 add_28 (Add)                   (None, 19, 19, 1024  0           ['add_27[0][0]',                 
                                )                                 'conv2d_87[0][0]']              
                                                                                                  
 avg_pool (GlobalAveragePooling  (None, 1024)        0           ['add_28[0][0]']                 
 2D)                                                                                              
                                                                                                  
 fc1000 (Dense)                 (None, 361)          370025      ['avg_pool[0][0]']               
          

In [6]:
callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                                patience=3, verbose=0, mode='min'
                                 , restore_best_weights=True)
callback2 = tf.keras.callbacks.ModelCheckpoint('./models/model_kyu_v2_b64_f256_l100_cor_{epoch:02d}_{val_accuracy:.4f}.h5', 
                                            monitor='val_accuracy', 
                                            verbose=0, save_best_only=False, save_weights_only=False, 
                                            mode='max', save_freq="epoch")

In [7]:
# model = load_model('./models/model_kyu_f256_l88_mir_02_0.0035.h5')

In [7]:
history = model.fit(
    dataset,
    epochs = 500,
#     steps_per_epoch = train_steps,
    validation_data = dataset_valid,
#     validation_steps = val_steps,
    callbacks = [callback1, callback2]
)

Epoch 1/500
169592/169592 [==============================] - 56134s 331ms/step - loss: 2.1308 - accuracy: 0.4815 - val_loss: 1.8280 - val_accuracy: 0.5184
Epoch 2/500
169592/169592 [==============================] - 64554s 381ms/step - loss: 1.7749 - accuracy: 0.5265 - val_loss: 1.7449 - val_accuracy: 0.5299
Epoch 3/500
169592/169592 [==============================] - 57578s 340ms/step - loss: 1.7047 - accuracy: 0.5375 - val_loss: 1.7188 - val_accuracy: 0.5329
Epoch 4/500
169592/169592 [==============================] - 60189s 355ms/step - loss: 1.6655 - accuracy: 0.5440 - val_loss: 1.7067 - val_accuracy: 0.5354
Epoch 5/500
169592/169592 [==============================] - ETA: 0s - loss: 1.6411 - accuracy: 0.5483

KeyboardInterrupt: 

In [8]:
model.save('./models/model_kyu_v2_b64_f256_l100_cor_5.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!